In [8]:
url_list = [
'https://www.naturzgroup.com/',  # 60
'https://kayalfoods.in/',  # 4556
'https://www.torresyribelles.com/home2'
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


In [49]:
file_path = "/Users/yoonhae/Downloads/crawling_processed_(데이터팀 검토) 그레인스캐너 패커 크롤링 정리 - Gulfood_정리포맷_20240123_134035.csv"
import pandas as pd

df = pd.read_csv(file_path)

In [85]:
import requests
from bs4 import BeautifulSoup

#url = url_list[1]
url = 'https://www.daviscl.com/'
response = requests.get(url, headers=headers, timeout=5, verify=False)
if response.status_code == 200:
    page_soup = BeautifulSoup(response.content, 'html.parser')

print(response.status_code)

/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.daviscl.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200


In [83]:
from bs4 import BeautifulSoup, NavigableString, Tag, Comment, Doctype, ProcessingInstruction


# 분할 기준 길이 설정 (예: 2000 토큰)
split_length = 2000

# 각 태그의 자식 노드의 텍스트를 추출하고 분할
split_texts = []
current_list = []
total_len = 0

for element in page_soup.descendants:  # body 태그의 자식 노드 순회
    if isinstance(element, NavigableString) and not isinstance(element, (Comment, Doctype, ProcessingInstruction)):
        txt = element.strip().replace('\n\n\n', ' ')
        parent = element.parent
        if txt and (parent and parent.name not in ['script', 'style', 'meta', 'noscript']):
            current_list.append(txt)
            total_len += len(txt)
    elif isinstance(element, Tag):
        if element.name == 'div':
            if total_len > split_length:
                split_texts.append('\n'.join(current_list))
                current_list.clear()
                total_len = 0

# 마지막 남은 텍스트 추가
if current_list:
    split_texts.append('\n'.join(current_list))

# 분할된 텍스트 출력
for i, text in enumerate(split_texts, 1):
    print(f"분할된 텍스트 {i}: {text[:100]}")
    print('len:', len(text))

page_txt = page_soup.text.strip().replace('\n\n\n', ' ')

분할된 텍스트 1: Davis Commodities Limited | sugar rice oil and fats distributor
top of page
HOME
ABOUT
PRODUCTS
Suga
len: 2157
분할된 텍스트 2: Quality over Quantity
We believe that quality of product and services are the first and foremost cri
len: 447


In [84]:
print('  ********* start!!')
print(split_texts[0], split_texts[1])
print('  ********* end!!')
print(page_txt)

  ********* start!!
Davis Commodities Limited | sugar rice oil and fats distributor
top of page
HOME
ABOUT
PRODUCTS
Sugar
Rice
Oils and Fats
SERVICES
CONTACT
INVESTORS
Davis Commodities Limited
​
From Mills to the Meals
A Singapore agri-commodity company with a reach not only to the Asia but to the rest of the world.
Learn More
Professional
Services
How can we help you?
Agri-COmmodity
Our speciality in containerized shipment of Sugar, Rice and Oil will best fulfil on your requirements.
Global Network
Our well connected network globally enables us to deliver and fulfil our customers’ needs efficiently.
Retail Marketing
Choosing right distribution channels and marketing strategies will maximize the potential of your product.
Features
&
Product Line
Some of the key quality commodities that we supply to our customers
Sugar
Up
Rice
Up
Oil
Up
At Davis Commodities Limited, we are committed to providing our customers with delicious, safe and healthy food products. We have an extensive producti

In [60]:
from bs4 import BeautifulSoup, NavigableString, Tag, Comment, Doctype

def get_total_length(page_soup):
    total_len = 0

    for element in page_soup.descendants:  # body 태그의 자식 노드 순회
        if isinstance(element, NavigableString) and not isinstance(element, (Comment, Doctype)):
            txt = element.strip().replace('\n\n\n', ' ')
            parent = element.parent
            if txt and (parent and parent.name not in ['script', 'style', 'meta', 'noscript']):
                total_len += len(txt)
    return total_len


def extract_soup(url):
    import requests

    soup = None
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(e)
        return 500, None

    return response.status_code, soup

In [ ]:

count = 0
for url in df['홈페이지 주소']:  # 45
    if not pd.isnull(url) and not pd.isna(url):
        code, page_soup = extract_soup(url)
        if code == 200:
            new_len = get_total_length(page_soup)
            txt_len = len(page_soup.text.strip().replace('\n\n\n', ' '))
            print(f"[{code}] {url} - txt / new : {txt_len} / {new_len}")
            if new_len > txt_len:
                print("** check 필요!!!! -", url )
        else:
            print(f"[{code}] {url}")

In [111]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import requests, time

session = requests.Session()
retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))


In [123]:

from urllib.parse import urlparse

def change_protocol(url):
    # URL 파싱
    parsed_url = urlparse(url)
    # 프로토콜 추출
    protocol = parsed_url.scheme
    if protocol == 'http':
        return parsed_url._replace(scheme='https').geturl()
    elif protocol == 'https':
        return parsed_url._replace(scheme='http').geturl()

    return url


def recovery_url(trouble_url):
    import re
    # URL 패턴 정의
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_pattern, trouble_url)

    # 결과 출력
    result = set()
    for url in urls:
        result.add(url)

    print(f'    ** recovery url - from ({trouble_url}) / to ({result})')
    return result


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

def extract_page(url, is_recursive=False):
    # 페이지 콘텐츠를 가져옵니다.
    try :
        response = session.get(url, timeout=15, headers=headers, verify=False)
    except requests.ConnectionError as ce:
        if not is_recursive:
            url_list = recovery_url(url)
            for sub_url in url_list:
                code, response = extract_page(sub_url, is_recursive=True)
                if code == 200:
                    break
            return code, response
        else:
            print(f'  ** error ({url})-', str(ce))
            return 500, None
    except Exception as e:
        print(f'  ** error ({url})-', str(e))
        return 500, None
    
    if response.status_code == 403:
        if not is_recursive:
            return extract_page(change_protocol(url), is_recursive=True)
    return response.status_code, response

extract_page('http://www.Tesorodelrio.com / http://www.fas.com.tn')

    ** recovery url - from (http://www.Tesorodelrio.com / http://www.fas.com.tn) / to ({'http://www.Tesorodelrio.com', 'http://www.fas.com.tn'})


/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tesorodelrio.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'tesorodelrio.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'tesorodelrio.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/

(200, <Response [200]>)

In [116]:
# headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
# headers['Host'] = 'www.oliocostadoro.com'




In [124]:
import pandas as pd

# 기존 DataFrame 예시
result_df = pd.DataFrame({
    'column1': [1, 2, 3],
    'column2': ['a', 'b', 'c'],
    'column3': [4.5, 6.7, 8.9]
})

# 새로운 행 추가

result_df = result_df.append({"column1": 4, "column2": 'd', "column3": 9.0}, ignore_index=True)


/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_54158/1913807472.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({"column1": 4, "column2": 'd', "column3": 9.0}, ignore_index=True)


In [132]:


# 예시 사용
data_dict = {
    'homepage': True,
    'extract_url': True,
    'company_name': False,
    'mail': True,
    'phone': True,
    'address': True,
    'fax': True,
    'products': True,
    'error': True
}

column_list = ['homepage', 'extract_url', 'company_name']

check_all_columns_true = lambda data_dict, target_column_list: all(data_dict.get(column, False) for column in target_column_list)

check_all_columns_true(data_dict, column_list)

False